<a href="https://colab.research.google.com/github/duybluemind1988/Data-science/blob/master/Kaggle_project_credit_default_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base ML method

## Get data

In [33]:
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder,OrdinalEncoder
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import time
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report,roc_curve, auc,precision_recall_curve,f1_score,balanced_accuracy_score

In [3]:
import zipfile
path_to_zip_file='/content/drive/My Drive/Data/home-credit-default-risk.zip'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall()

In [21]:
# Training data
app_train = pd.read_csv('/content/application_train.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

Training data shape:  (307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,...,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,...,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
app_train.dtypes

SK_ID_CURR                      int64
TARGET                          int64
NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                   object
                               ...   
AMT_REQ_CREDIT_BUREAU_DAY     float64
AMT_REQ_CREDIT_BUREAU_WEEK    float64
AMT_REQ_CREDIT_BUREAU_MON     float64
AMT_REQ_CREDIT_BUREAU_QRT     float64
AMT_REQ_CREDIT_BUREAU_YEAR    float64
Length: 122, dtype: object

## Feature engineer

Label encoder object

In [23]:
categorical_features = app_train.select_dtypes(include=['object']).columns
for feature in categorical_features:
    #train[feature]=pd.Categorical(train[feature])
    app_train[feature]=str(app_train[feature].values)
    app_train[feature]=LabelEncoder().fit_transform(app_train[feature])

Label encoder numeric (for all numeric column <=7 unique value)

In [24]:
numeric_features = app_train.drop('TARGET',axis=1).select_dtypes(exclude=['object']).columns
for feature in numeric_features:
    if len(list(app_train[feature].unique())) <= 7:
        #train[feature]=pd.Categorical(train[feature])
        app_train[feature]=str(app_train[feature].values)
        app_train[feature]=LabelEncoder().fit_transform(app_train[feature])

In [25]:
app_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,0,0,0,0,0,0.018801,-9461,-637,-3648.0,-2120,NaN,0,0,0,0,0,0,0,1.0,0,0,0,10,0,0,0,0,0,0,...,0.0205,0.0193,0.0000,0.0000,0,0,0.0149,0,0,2.0,2.0,2.0,2.0,-1134.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0
1,100003,0,0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0,0,0,0,0,0.003541,-16765,-1188,-1186.0,-291,NaN,0,0,0,0,0,0,0,2.0,0,0,0,11,0,0,0,0,0,0,...,0.0787,0.0558,0.0039,0.0100,0,0,0.0714,0,0,1.0,0.0,1.0,0.0,-828.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,100004,0,0,0,0,0,0,67500.0,135000.0,6750.0,135000.0,0,0,0,0,0,0.010032,-19046,-225,-4260.0,-2531,26.0,0,0,0,0,0,0,0,1.0,0,0,0,9,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,0,0,NaN,0,0,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0,0,0,0,0,135000.0,312682.5,29686.5,297000.0,0,0,0,0,0,0.008019,-19005,-3039,-9833.0,-2437,NaN,0,0,0,0,0,0,0,2.0,0,0,0,17,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,0,0,NaN,0,0,2.0,0.0,2.0,0.0,-617.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,0,0,0,0,0,0.028663,-19932,-3038,-4311.0,-3458,NaN,0,0,0,0,0,0,0,1.0,0,0,0,11,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,0,0,NaN,0,0,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0,0,0,0,0,157500.0,254700.0,27558.0,225000.0,0,0,0,0,0,0.032561,-9327,-236,-8456.0,-1982,NaN,0,0,0,0,0,0,0,1.0,0,0,0,15,0,0,0,0,0,0,...,0.1509,0.2001,0.0757,0.1118,0,0,0.2898,0,0,0.0,0.0,0.0,0.0,-273.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
307507,456252,0,0,0,0,0,0,72000.0,269550.0,12001.5,225000.0,0,0,0,0,0,0.025164,-20775,365243,-4388.0,-4090,NaN,0,0,0,0,0,0,0,1.0,0,0,0,8,0,0,0,0,0,0,...,0.0205,0.0261,0.0000,0.0000,0,0,0.0214,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
307508,456253,0,0,0,0,0,0,153000.0,677664.0,29979.0,585000.0,0,0,0,0,0,0.005002,-14966,-7921,-6737.0,-5150,NaN,0,0,0,0,0,0,0,1.0,0,0,0,9,0,0,0,0,0,0,...,0.0855,0.9445,0.0000,0.0000,0,0,0.7970,0,0,6.0,0.0,6.0,0.0,-1909.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,0.0,1.0
30750

In [29]:
print(app_train['TARGET'].value_counts())
print(app_train['TARGET'].value_counts(normalize=True))

0    282686
1     24825
Name: TARGET, dtype: int64
0    0.919271
1    0.080729
Name: TARGET, dtype: float64


In [32]:
dataX = app_train.drop(['TARGET'],axis=1)
dataY = app_train['TARGET']

X_train, X_test, y_train, y_test = \
    train_test_split(dataX, dataY, test_size=0.30, \
                    random_state=2018, stratify=dataY)

In [ ]:
start=time.time()
xgb=XGBClassifier(
                nthread=16,
                objective='binary:logistic',
                eval_metric = 'logloss',
                #scale_pos_weight=1,
                enable_categorical=True,
                seed=2018,
                verbosity=1    
                              )
xgb.fit(X_train, y_train)
end=time.time()
print("time: ",end-start) # 65 s

In [37]:
y_test

141933    0
189485    1
54773     0
263712    0
269251    0
         ..
118382    1
103254    0
86339     0
44938     0
256529    0
Name: TARGET, Length: 92254, dtype: int64

In [40]:
y_pred_probs

array([[0.86486834, 0.13513164],
       [0.7159316 , 0.2840684 ],
       [0.97378576, 0.02621426],
       ...,
       [0.9145097 , 0.08549029],
       [0.83730584, 0.16269414],
       [0.93010855, 0.06989145]], dtype=float32)

In [35]:
y_pred=xgb.predict(X_test)
y_pred_probs=xgb.predict_proba(X_test)
print("f1 score",f1_score(y_test,y_pred))
print("balanced_accuracy_score:",balanced_accuracy_score(y_test,y_pred))
print("AUC:",auc(y_test,y_pred_probs))
print('classification_report \n',classification_report(y_test, y_pred))

f1 score 0.01224381155177003
balanced_accuracy_score: 0.502970161151022


ValueError: ignored

In [ ]:
start=time.time()
lgbm=LGBMClassifier(
    n_estimators = 200,
    boosting_type = 'gbdt',
    objective = 'binary',
    metric= 'binary:logloss',
    metric_freq=50,
    learning_rate=0.01,
    max_depth=4,
    num_leaves= 16, #(2^maxdepth)
    min_data_in_leaf=2000,
    scale_pos_weight=1,
    num_threads=16,
    random_state =500,
    bagging_freq=0,
                                )

lgbm.fit(X_train, y_train)
end=time.time() 
print("time: ",end-start) # 9s vs 33 s H2o light gbm

y_pred=lgbm.predict(X_test)
y_pred_probs=lgbm.predict_proba(X_test)

print("f1 score",f1_score(y_test,y_pred))
print("balanced_accuracy_score:",balanced_accuracy_score(y_test,y_pred))
print("AUC:",auc(y_test,y_pred))
print('classification_report \n',classification_report(y_test, y_pred))
